In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#單字 one-hot encoding (簡易版)
samples = ['The cat sat on the mat.', 'The dog ate my homework.'] #初始資料: 此list每個元素是一個樣本

token_index = {} #建立空字典，用來儲存所有tokens及其編號(鍵值)

#建立字典
for sample in samples: #取字串
    for word in sample.split(): #對字串進行分詞，注意:真實案例中還需移除標點符號及特殊字元
        if not word in token_index:
            token_index[word] = len(token_index) + 1 #以新字token為鍵，以目前詞彙字典長度+1作為該鍵的值，加入字典
                                                     #0有特殊用途，不指定給任何文字

#token vectorizing(向量化)
max_length = 10 #只處理每個輸入字串樣本中前10個單字

results = np.zeros((len(samples), max_length, max(token_index.values()) + 1)) #(samples, tokens, index) index從1開始
print(results.shape)

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

(2, 10, 11)


In [3]:
print(results)

[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]


In [4]:
#字元 one-hot encoding (簡易版)
import string 

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable #所有可印出的ASCII字元的字串 '0123456789abc...'
print(len(characters)) #共100種字元(tokens)

token_index = dict(zip(characters, range(1, len(characters) + 1)))

max_length = 50 #只處理樣本中前50個字元

results = np.zeros((len(samples), max_length, max(token_index.values()) + 1)) #(samples, tokens, index) index從1開始
print(results.shape)

for i, sample in enumerate(samples):
    for j, character in enumerate(sample):
        index = token_index.get(character)
        results[i, j, index] = 1.

100
(2, 50, 101)


In [5]:
print(results)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [6]:
#用Keras做單字的one-hot encoding
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer #匯入Keras分詞器

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

tokenizer = Tokenizer(num_words=1000) #建立分詞器(處理前1000個最常用單字)
tokenizer.fit_on_texts(samples) #建立字典，鍵值依出現頻率及順序決定，0不使用
sequences = tokenizer.texts_to_sequences(samples) #將samples裡每個單字用字典轉換成整數鍵值，並以list呈現
print(sequences)

one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary') #若不指定mode，預設是'binary'，也就是one-hot encoding

print(one_hot_results.shape)

#更適合稱作 Muti-hot encoding
print(one_hot_results[0][:15])
print(one_hot_results[1][:15])

word_index = tokenizer.word_index #取得單字與索引之間的對應關聯
print(word_index)

print(f"Found {len(word_index)} unique tokens")

[[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
(2, 1000)
[0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
{'the': 1, 'cat': 2, 'sat': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}
Found 9 unique tokens


In [7]:
#one-hot hashing trick(雜湊技巧)
#當字典中的tokens數量太大時，將token雜湊成固定長度的向量
#優點: 可節省記憶體，並允許再現編輯
#缺點: 可能發生雜湊碰撞(hash collisions)，即兩個不同token擁有相同的雜湊值，當雜湊空間維度遠大於被雜湊之tokens總數時，雜湊碰撞可能性隨之降低

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

dimensionality = 1000 #將token雜湊成大小為1000的向量。若有接近或大於1000個單字，則會造成許多雜湊碰撞，使此編碼準確性降低
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = abs(hash(word)) % dimensionality #將token雜湊成0~1000之間的隨機整數索引
        results[i, j, index] = 1.
print(results.shape)

(2, 10, 1000)
